In [2]:
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, InputLayer, Activation
from keras.optimizers import Adam

import tensorflow as tf
import numpy as np

import nltk
nltk.download('brown')

import gensim
import gensim.downloader
from nltk.corpus import brown
import string

import nltk
nltk.download('punkt')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Model

In [3]:
df = pd.read_json("train.json")

In [4]:
df_test = pd.read_json("test.json")

In [5]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle

In [6]:
import tensorflow_hub as hub
import tensorflow as tf



In [ ]:
class feed_forwrd():
    def __init__(self, X_train,y_train):
        self.X_train=X_train
        self.y_train=y_train
    
    def build_ff_nn(self):
        self.model = Sequential()
        self.model.add(InputLayer(input_shape=(100,)))
        self.model.add(Dense(len(np.unique(self.output_class))))
        self.model.add(Activation('softmax'))
        self.model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
        self.model.summary()
    def train_ff_nn(self):
        tag2int = {tag: i for i, tag in enumerate(list(np.unique(self.output_class)))}
        y_int=[]
        #print(self.output_class)
        for tag in self.y_train:
            y_int.append(tag2int[tag])
        y_train_ = tf.keras.utils.to_categorical(y_int,num_classes=len(tag2int))
        
        self.history = self.model.fit(self.X_train, (y_train_), batch_size=64, epochs=50,validation_split=0.2)
        
    def prediction(self, X_test):
        
        y_pred = self.model.predict(X_test)
        y_pred_tag=[]
        int2tag = {i: tag for i, tag in enumerate(list(np.unique(self.y_train)))}
        for i in y_pred:
            max_index = np.argmax(i)
            tag_pred = int2tag[max_index]
            y_pred_tag.append(tag_pred)
        return y_pred_tag

In [7]:
!pip install transformers

     |████████████████████████████████| 2.0MB 5.0MB/s 
     |████████████████████████████████| 3.2MB 34.4MB/s 
     |████████████████████████████████| 890kB 35.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=cb4a7dcbb2eefb132af7cb1e25c708e21903f1007b836a636ea7994e8bbe690b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [8]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel



In [9]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')
#input_ids = tf.constant(tokenizer.encode("Hello, my dog is cute"))[None, :]  # Batch size 1
#outputs = model(input_ids)
#last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [27]:
X_train=[]
for i in range(len(df['context'])):
  #if i%10 == 0:
  #print(i)
  context_emb = np.zeros(768)
  for j in range(len(df['context'][i])):
      input_ids = tf.constant(tokenizer.encode(df['context'][i][j]))[None, :]  # Batch size 1
      if len(input_ids[0])>511:
        input_ids = input_ids[:,:512]
      outputs = model(input_ids)
      last_hidden_states = outputs[0]
      try:
        index = tokenizer.tokenize(df['context'][i][j]).index(df['nc'][i].split()[0])
        context_emb += (last_hidden_states[0][index+1]+last_hidden_states[0][index+2])/2
      except:
        index=0
        context_emb += (last_hidden_states[0][0]+last_hidden_states[0][0])/2
  if len(df['context'][i])!=0:
    context_emb=context_emb/len(df['context'][i])
  X_train.append(context_emb)

KeyboardInterrupt: ignored

In [29]:
y_train = df['label']

In [ ]:
M = feed_forwrd(df)
M.build_ff_nn()
M.get_embedding()
M.train_ff_nn()
#y_pred = M.predict(preprocessed_test_reviews)

In [31]:
X_test=[]
for i in range(len(df_test['context'])):
  #if i%10 == 0:
  #print(i)
  context_emb = np.zeros(768)
  for j in range(len(df_test['context'][i])):
      input_ids = tf.constant(tokenizer.encode(df_test['context'][i][j]))[None, :]  # Batch size 1
      if len(input_ids[0])>511:
        input_ids = input_ids[:,:512]
      outputs = model(input_ids)
      last_hidden_states = outputs[0]
      try:
        index = tokenizer.tokenize(df_test['context'][i][j]).index(df_test['nc'][i].split()[0])
        context_emb += (last_hidden_states[0][index+1]+last_hidden_states[0][index+2])/2
      except:
        index=0
        context_emb += (last_hidden_states[0][0]+last_hidden_states[0][0])/2
  if len(df_test['context'][i])!=0:
    context_emb=context_emb/len(df_test['context'][i])
  X_test.append(context_emb)

In [ ]:
M.prediction(X_test)

In [ ]:
df_test = pd.DataFrame(columns=['nc','context','label'])

In [ ]:
df_test['nc']=df3['nc']
df_test['context']=df3['context']
df_test['label']=y_pred



In [ ]:
df_test.to_json('test_label_part2.json')